Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Model

In [5]:
batch_size = 128

graph = tf.Graph()

with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                                 logits=logits) + \
                         0.001 * tf.nn.l2_loss(weights))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
    
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_predictions = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    
with tf.Session(graph=graph) as sess:
    
    tf.global_variables_initializer().run()
    
    print('Initialized')
    
    for i in range(3001):
        
        offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
        
        X = train_dataset[offset:(offset + batch_size), :]
        y = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: X,
            tf_train_labels: y
        }
        
        _, l, predictions = sess.run([optimizer, loss, train_predictions],
                                    feed_dict=feed_dict)
        
        if i % 500 == 0:
            
            print('Step {:d}'.format(i))
            print('Training Loss: {:0.1f}'.format(l))
            print('Training Accuracy: {:0.1f}%'.format(accuracy(predictions, y)))
            print('Validation Accuracy: {:0.1f}%'.format(accuracy(valid_predictions.eval(),
                                                                valid_labels)))
            print('')
            
    
    print('Test Accuracy: {:0.1f}%'.format(accuracy(test_predictions.eval(),
                                                 test_labels)))

Initialized
Step 0
Training Loss: 22.4
Training Accuracy: 7.0%
Validation Accuracy: 9.4%

Step 500
Training Loss: 2.7
Training Accuracy: 76.6%
Validation Accuracy: 76.1%

Step 1000
Training Loss: 1.6
Training Accuracy: 80.5%
Validation Accuracy: 78.2%

Step 1500
Training Loss: 1.1
Training Accuracy: 82.0%
Validation Accuracy: 79.1%

Step 2000
Training Loss: 0.9
Training Accuracy: 83.6%
Validation Accuracy: 81.0%

Step 2500
Training Loss: 0.9
Training Accuracy: 79.7%
Validation Accuracy: 81.5%

Step 3000
Training Loss: 0.9
Training Accuracy: 78.9%
Validation Accuracy: 82.1%

Test Accuracy: 89.1%


## Neural network with 1 hidden layer

In [1]:
batch_size = 128

# number of neurons in the hidden layer
num_hidden_layer = 1000


graph = tf.Graph()

with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_layer]))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    logits2 = tf.matmul(logits1, weights2) + biases2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                                 logits=logits2) + \
                         0.001 * tf.nn.l2_loss(weights1) + 0.001 * tf.nn.l2_loss(weights2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
    
    train_predictions = tf.nn.softmax(logits2)
    
    # valid_predictions
    logits1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    logits2 = tf.matmul(logits1, weights2) + biases2
    valid_predictions = tf.nn.softmax(logits2)
    
    # test_predictions
    logits1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    logits2 = tf.matmul(logits1, weights2) + biases2
    test_predictions = tf.nn.softmax(logits2)
    
    
with tf.Session(graph=graph) as sess:
    
    tf.global_variables_initializer().run()
    
    print('Initialized')
    
    for i in range(3001):
        
        offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
        
        X = train_dataset[offset:(offset + batch_size), :]
        y = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: X,
            tf_train_labels: y
        }
        
        _, l, predictions = sess.run([optimizer, loss, train_predictions],
                                    feed_dict=feed_dict)
        
        if i % 500 == 0:
            
            print('Step {:d}'.format(i))
            print('Training Loss: {:0.1f}'.format(l))
            print('Training Accuracy: {:0.1f}%'.format(accuracy(predictions, y)))
            print('Validation Accuracy: {:0.1f}%'.format(accuracy(valid_predictions.eval(),
                                                                valid_labels)))
            print('')
            
    
    print('Test Accuracy: {:0.1f}%'.format(accuracy(test_predictions.eval(),
                                                 test_labels)))

NameError: name 'tf' is not defined

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

The following demonstrates overfitting in a simple logistic regression scenario

In [7]:
batch_size = 128

graph = tf.Graph()

with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                                 logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
    
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_predictions = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    
with tf.Session(graph=graph) as sess:
    
    tf.global_variables_initializer().run()
    
    print('Initialized')
    
    for i in range(3001):
        
        # restrict training data to a few batches
        offset = np.random.choice([1, 2, 3, 4])
        
        X = train_dataset[offset:(offset + batch_size), :]
        y = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: X,
            tf_train_labels: y
        }
        
        _, l, predictions = sess.run([optimizer, loss, train_predictions],
                                    feed_dict=feed_dict)
        
        if i % 500 == 0:
            
            print('Step {:d}'.format(i))
            print('Training Loss: {:0.1f}'.format(l))
            print('Training Accuracy: {:0.1f}%'.format(accuracy(predictions, y)))
            print('Validation Accuracy: {:0.1f}%'.format(accuracy(valid_predictions.eval(),
                                                                valid_labels)))
            print('')
            
    
    print('Test Accuracy: {:0.1f}%'.format(accuracy(test_predictions.eval(),
                                                 test_labels)))

Initialized
Step 0
Training Loss: 17.4
Training Accuracy: 7.0%
Validation Accuracy: 13.1%

Step 500
Training Loss: 0.0
Training Accuracy: 100.0%
Validation Accuracy: 51.3%

Step 1000
Training Loss: 0.0
Training Accuracy: 100.0%
Validation Accuracy: 52.3%

Step 1500
Training Loss: 0.0
Training Accuracy: 100.0%
Validation Accuracy: 52.9%

Step 2000
Training Loss: 0.0
Training Accuracy: 100.0%
Validation Accuracy: 53.2%

Step 2500
Training Loss: 0.0
Training Accuracy: 100.0%
Validation Accuracy: 53.5%

Step 3000
Training Loss: 0.0
Training Accuracy: 100.0%
Validation Accuracy: 53.7%

Test Accuracy: 59.8%


The test accuracy is not very good

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128

# number of neurons in the hidden layer
num_hidden_layer = 1000


graph = tf.Graph()

with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_layer]))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    logits2 = tf.matmul(tf.nn.dropout(logits1, 0.5), weights2) + biases2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                                 logits=logits2) + \
                         0.001 * tf.nn.l2_loss(weights1) + 0.001 * tf.nn.l2_loss(weights2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
    
    train_predictions = tf.nn.softmax(logits2)
    
    # valid_predictions
    logits1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    logits2 = tf.matmul(logits1, weights2) + biases2
    valid_predictions = tf.nn.softmax(logits2)
    
    # test_predictions
    logits1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    logits2 = tf.matmul(logits1, weights2) + biases2
    test_predictions = tf.nn.softmax(logits2)
    
    
with tf.Session(graph=graph) as sess:
    
    tf.global_variables_initializer().run()
    
    print('Initialized')
    
    for i in range(3001):
        
        offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
        
        X = train_dataset[offset:(offset + batch_size), :]
        y = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: X,
            tf_train_labels: y
        }
        
        _, l, predictions = sess.run([optimizer, loss, train_predictions],
                                    feed_dict=feed_dict)
        
        if i % 500 == 0:
            
            print('Step {:d}'.format(i))
            print('Training Loss: {:0.1f}'.format(l))
            print('Training Accuracy: {:0.1f}%'.format(accuracy(predictions, y)))
            print('Validation Accuracy: {:0.1f}%'.format(accuracy(valid_predictions.eval(),
                                                                valid_labels)))
            print('')
            
    
    print('Test Accuracy: {:0.1f}%'.format(accuracy(test_predictions.eval(),
                                                 test_labels)))

Initialized
Step 0
Training Loss: 723.3
Training Accuracy: 10.9%
Validation Accuracy: 32.5%

Step 500
Training Loss: 203.0
Training Accuracy: 69.5%
Validation Accuracy: 81.2%

Step 1000
Training Loss: 118.3
Training Accuracy: 77.3%
Validation Accuracy: 80.6%

Step 1500
Training Loss: 69.0
Training Accuracy: 72.7%
Validation Accuracy: 82.5%

Step 2000
Training Loss: 40.9
Training Accuracy: 82.8%
Validation Accuracy: 83.3%

Step 2500
Training Loss: 24.8
Training Accuracy: 79.7%
Validation Accuracy: 84.4%

Step 3000
Training Loss: 15.4
Training Accuracy: 80.5%
Validation Accuracy: 85.2%

Test Accuracy: 92.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [9]:
batch_size = 128

# number of neurons in the hidden layers
num_hidden_layer1 = 1000
num_hidden_layer2 = 500
num_hidden_layer3 = 250


graph = tf.Graph()

with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    global_step = tf.Variable(0)

    
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_layer1],
                                              stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([num_hidden_layer1]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_layer1, num_hidden_layer2],
                                              stddev=np.sqrt(2.0 / num_hidden_layer1)))
    biases2 = tf.Variable(tf.zeros([num_hidden_layer2]))
    
    weights3 = tf.Variable(tf.truncated_normal([num_hidden_layer2, num_hidden_layer3],
                                              stddev=np.sqrt(2.0 / num_hidden_layer2)))
    biases3 = tf.Variable(tf.zeros([num_hidden_layer3]))
    
    weights4 = tf.Variable(tf.truncated_normal([num_hidden_layer3, num_labels],
                                              stddev=np.sqrt(2.0 / num_hidden_layer3)))
    biases4 = tf.Variable(tf.zeros([num_labels]))
    
    logits1 = tf.nn.relu(tf.matmul(tf.nn.dropout(tf_train_dataset, 0.8), weights1) + biases1)
    logits2 = tf.nn.relu(tf.matmul(tf.nn.dropout(logits1, 0.8), weights2) + biases2)
    logits3 = tf.nn.relu(tf.matmul(tf.nn.dropout(logits2, 0.8), weights3) + biases3)
    logits4 = tf.matmul(logits3, weights4) + biases4

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits4))
    
    learning_rate = tf.train.exponential_decay(0.5, global_step, 2000, 0.3, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_predictions = tf.nn.softmax(logits4)
    
    # valid_predictions
    logits1_pred = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    logits2_pred = tf.nn.relu(tf.matmul(logits1_pred, weights2) + biases2)
    logits3_pred = tf.nn.relu(tf.matmul(logits2_pred, weights3) + biases3)
    logits4_pred = tf.matmul(logits3_pred, weights4) + biases4
    valid_predictions = tf.nn.softmax(logits4_pred)
    
    # test_predictions
    logits1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    logits2 = tf.nn.relu(tf.matmul(logits1, weights2) + biases2)
    logits3 = tf.nn.relu(tf.matmul(logits2, weights3) + biases3)
    logits4 = tf.matmul(logits3, weights4) + biases4
    test_predictions = tf.nn.softmax(logits4)
    
    
with tf.Session(graph=graph) as sess:
    
    tf.global_variables_initializer().run()
    
    print('Initialized')
    
    for i in range(3001):
        
        offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
        
        X = train_dataset[offset:(offset + batch_size), :]
        y = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset: X,
            tf_train_labels: y
        }
        
        _, l, predictions = sess.run([optimizer, loss, train_predictions],
                                    feed_dict=feed_dict)
        
        if i % 500 == 0:
            
            print('Step {:d}'.format(i))
            print('Training Loss: {:0.1f}'.format(l))
            print('Training Accuracy: {:0.1f}%'.format(accuracy(predictions, y)))
            print('Validation Accuracy: {:0.1f}%'.format(accuracy(valid_predictions.eval(),
                                                                valid_labels)))
            print('')
            
    
    print('Test Accuracy: {:0.1f}%'.format(accuracy(test_predictions.eval(),
                                                 test_labels)))

Initialized
Step 0
Training Loss: 2.3
Training Accuracy: 16.4%
Validation Accuracy: 10.4%

Step 500
Training Loss: 0.5
Training Accuracy: 81.2%
Validation Accuracy: 84.9%

Step 1000
Training Loss: 0.4
Training Accuracy: 89.1%
Validation Accuracy: 86.7%

Step 1500
Training Loss: 0.4
Training Accuracy: 87.5%
Validation Accuracy: 87.2%

Step 2000
Training Loss: 0.4
Training Accuracy: 88.3%
Validation Accuracy: 88.4%

Step 2500
Training Loss: 0.4
Training Accuracy: 88.3%
Validation Accuracy: 88.8%

Step 3000
Training Loss: 0.4
Training Accuracy: 85.2%
Validation Accuracy: 89.3%

Test Accuracy: 94.7%
